## Preparación del entorno

In [1]:
# Importar bibliotecas necesarias para el análisis
import numpy as np
import pandas as pd

## Carga de datos

### Delitos en la Ciudad
El dataset utilizado en este análisis proviene del [Portal de Datos Abiertos del Gobierno de la Ciudad de Buenos Aires](https://data.buenosaires.gob.ar/dataset/delitos). El dataset abarca el período 2016-2023 Y los datos están organizados en archivos separados por año, donde cada archivo contiene la información de los delitos reportados en ese período específico. Los datos están disponibles bajo la licencia Creative Commons Attribution.

El dataset incluye las siguientes variables:

|Nombre|Tipo|Descripción|
|------|----|-----------|
|id-mapa o id-sum|integer|Identificador único.|
|anio|date|Año en el que se registró el evento.|
|mes|string|Mes en que ocurrió el evento.|
|dia|string|Día de la semana en que ocurrió el evento.|
|fecha|date|Fecha exacta del evento.|
|franja|integer|Franja horaria en la que ocurrió el evento.|
|tipo|string|Clasificación del tipo de delito.|
|subtipo|string|Subtipo del delito, más específico.|
|uso_arma|boolean|Indicador de uso de arma.|
|uso_moto|boolean|Indicador de uso de moto en el evento.|
|barrio|string|Barrio donde ocurrió el evento.|
|comuna|integer|Comuna donde ocurrió el evento.|
|latitud|float|Latitud geográfica donde ocurrió el evento.|
|longitud|float|Longitud geográfica donde ocurrió el evento.|
|cantidad|integer|Número de eventos registrados en esa ubicación y fecha.|

In [ ]:
# Lista de archivos que hay que importar (delitos desde 2016 hasta 2023)
# Los archivos están almacenados en la misma carpeta que el notebook Jupyter
archivos_csv = [
    "delitos_2016.csv",
    "delitos_2017.csv",
    "delitos_2018.csv",
    "delitos_2019.csv",
    "delitos_2020.csv",
    "delitos_2021.csv",
    "delitos_2022.csv",
    "delitos_2023.csv"
]

# Crear lista para almacenar todos los data frames
dfs = []

# Importar los archivos de delitos uno por uno
for arch in archivos_csv:
    try:
        if arch != "delitos_2023.csv": #El arch de 2023 utiliza ";" como separadores, en lugar de ","
            df = pd.read_csv(arch)
        else:
            df = pd.read_csv(arch, sep=";")
        dfs.append(df)
    except FileNotFoundError:
        print(f"Advertencia: El archivo {arch} no fue encontrado y será omitido.")
    except Exception as e:
        print(f"Error al leer el archivo {arch}: {e}")

# Crear el data frame final de delitos si se cargaron datos
if dfs:
    df_delitos = pd.concat(dfs, ignore_index=True)
    print(f"Se cargaron {len(dfs)} archivos. Total de registros: {len(df_delitos)}")
    # Mostrar primeras filas del data frame de delitos
    display(df_delitos.head())
else:
    print("No se cargó ningún archivo de datos. Verifique la ruta y los nombres de los archivos.")
    # Crear un DataFrame vacío para evitar errores posteriores
    df_delitos = pd.DataFrame()

,id-mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad,id-sum
0,500001.0,2016,ENERO,MARTES,2016-01-26,21.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34.617668,-58.530961,1,NaN
1,500004.0,2016,ENERO,MIERCOLES,2016-01-20,16.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34.620262,-58.530738,1,NaN
2,500007.0,2016,ENERO,DOMINGO,2016-01-03,13.0,Robo,Robo total,SI,NO,LINIERS,9.0,-34.640094,-58.529826,1,NaN
3,500010.0,2016,ENERO,SABADO,2016-01-09,17.0,Robo,Robo total,NO,NO,LINIERS,9.0,-34.640094,-58.529826,1,NaN
4,500013.0,2016,ENERO,LUNES,2016-01-25,18.0,Robo,Robo total,NO,NO,LINIERS,9.0,-34.640094,-58.529826,1,NaN


### Población de la Ciudad
Para calcular la métrica de "cantidad de delitos por 100.000 personas", es necesario conocer la población de la Ciudad de Buenos Aires a lo largo de los años. Esta información se obtuvo del Instituto de Estadística y Censos de la Ciudad Autónoma de Buenos Aires. Se puede encontrar la información en el siguiente enlace: [Población total estimada por sexo, superficie y densidad poblacional según comuna. Ciudad de Buenos Aires. Años 2006/2023](https://www.estadisticaciudad.gob.ar/eyc/?p=76599)

In [3]:
# Crear diccionario que va a almacenar la población de la Ciudad en cada año
poblacion_ciudad = {
    2016: 3059122,
    2017: 3063728,
    2018: 3068043,
    2019: 3072029,
    2020: 3075646,
    2021: 3078836,
    2022: 3081550,
    2023: 3083770
}

## Limpieza de datos

### Limpieza de columnas id-mapa y id-sum

In [ ]:
# Algunos archivos utilizan "id-mapa" como id y otros utilizan "id-sum".
# Crear columna que almacene los ids de los delitos reportados
if 'id' not in df_delitos.columns:
    df_delitos['id'] = df_delitos['id-mapa'].fillna(df_delitos['id-sum'])

Ids faltantes: 0


In [ ]:
# Verificar que no halla ids faltantes
print(f"Ids faltantes: {df_delitos['id'].isna().sum()}")

In [ ]:
# Eliminar columnas "id-mapa" y "id-sum"
if 'id-mapa' in df_delitos.columns:
    df_delitos.drop('id-mapa', axis=1, inplace=True)
if 'id-sum' in df_delitos.columns:
    df_delitos.drop('id-sum', axis=1, inplace=True)

### Limpieza de la columna "anio":

In [5]:
# Verificar que los años son correctos
anios_invalidos = df_delitos[(df_delitos["anio"] < 2016) | (df_delitos["anio"] > 2023)]
print(f"cantidad de anios invalidos: {len(anios_invalidos)}")

cantidad de anios invalidos: 0
tipo de dato: int64
nuevo tipo de dato: int16


In [ ]:
# Verificar el tipo de dato de la columna
print(f"tipo de dato: {df_delitos["anio"].dtype}")

In [ ]:
# Convertir en un tipo de entero más pequeño
df_delitos["anio"] = df_delitos["anio"].astype("int16")
print(f"nuevo tipo de dato: {df_delitos["anio"].dtype}")

### Limpieza de la columna "mes":

In [6]:
# Verificar que los valores son válidos
print("meses:", end=" ")
for mes in df_delitos["mes"].unique():
    print(mes, end=" ")
print()

meses: ENERO FEBRERO MARZO ABRIL MAYO JUNIO JULIO AGOSTO SEPTIEMBRE OCTUBRE NOVIEMBRE DICIEMBRE 
tipo de dato: object
nuevo tipo de dato: string


In [ ]:
# Verificar tipo de dato
print(f"tipo de dato: {df_delitos["mes"].dtype}")

In [ ]:
# Convertir a tipo de dato correcto
df_delitos["mes"] = df_delitos["mes"].astype("string")
print(f"nuevo tipo de dato: {df_delitos["mes"].dtype}")

### Limpieza de la columna "dia":

In [7]:
# Verificar que los días de la semana sean válidos
print("dias de la semana:", end=" ")
for dia in df_delitos["dia"].unique():
    print(dia, end=" ")
print()

dias de la semana: MARTES MIERCOLES DOMINGO SABADO LUNES VIERNES JUEVES SÁBADO MIÉRCOLES 
dias de la semana corregidos: MARTES MIERCOLES DOMINGO SABADO LUNES VIERNES JUEVES 
tipo de dato: object
nuevo tipo de dato: string


In [ ]:
# Se detectó un error en los datos: Los días "SÁBADO" y "MIÉRCOLES" aparecen con y sin tilde
# Para solucionar esto, se normalizan los valores eliminado los tildes para mantener la consistencia
df_delitos.loc[df_delitos["dia"] == "SÁBADO", "dia"] = "SABADO"
df_delitos.loc[df_delitos["dia"] == "MIÉRCOLES", "dia"] = "MIERCOLES"

In [ ]:
# Verificar los cambios
print("dias de la semana corregidos:", end=" ")
for dia in df_delitos["dia"].unique():
    print(dia, end=" ")
print()

In [ ]:
# Verificar el tipo de dato
print(f"tipo de dato: {df_delitos["dia"].dtype}")

In [ ]:
# Coventir al tipo de dato correcto
df_delitos["dia"] = df_delitos["dia"].astype("string")
print(f"nuevo tipo de dato: {df_delitos["dia"].dtype}")

### Limpieza de la columna "fecha":

In [8]:
# Verificar el tipo de dato
print(f"tipo de dato: {df_delitos["fecha"].dtype}")

tipo de dato: object
nuevo tipo de dato: datetime64[ns]
cantidad de fechas fuera de rango: 0


In [ ]:
# Convertir la columna fecha al tipo de dato correcto para facilitar el análisis
df_delitos["fecha"] = pd.to_datetime(df_delitos["fecha"], format="mixed")
print(f"nuevo tipo de dato: {df_delitos["fecha"].dtype}")

In [ ]:
# Verificar que las fechas esten en el rango correcto
fechas_invalidas = df_delitos[
    ~(df_delitos['fecha'].dt.year.between(2016, 2023))
]
print(f"cantidad de fechas fuera de rango: {len(fechas_invalidas)}")

### Limpieza de la columna "franja"

In [9]:
# Verificar que los valores son válidos (0-23)
franjas_invalidas = df_delitos[(df_delitos["franja"] < 0) | (df_delitos["franja"] > 23)]
print(f"cantidad de franjas invalidas: {len(franjas_invalidas)}")

cantidad de franjas invalidas: 0
2328
tipo de dato: float64
nuevo tipo de dato: Int8


In [ ]:
# Verificar si hay valores faltantes
val_faltantes = df_delitos["franja"].isnull().sum()
print(val_faltantes)

In [ ]:
# Verificar tipo de dato
print(f"tipo de dato: {df_delitos["franja"].dtype}")

In [ ]:
# Convertir al tipo de dato correcto
df_delitos["franja"] = df_delitos["franja"].astype("Int8")
print(f"nuevo tipo de dato: {df_delitos["franja"].dtype}")

### Limpieza de la columna "tipo":

In [10]:
# Verificar que los valores sean válidos
print(f"tipos de delitos: {df_delitos["tipo"].unique()}")

tipos de delitos: ['Robo' 'Hurto' 'Vialidad' 'Homicidios' 'Amenazas' 'Lesiones']
tipo de dato: object
nuevo tipo de dato: string


In [ ]:
# Verificar tipo de dato
print(f"tipo de dato: {df_delitos["tipo"].dtype}")

In [ ]:
# Convertir al tipo de dato correcto
df_delitos["tipo"] = df_delitos["tipo"].astype("string")
print(f"nuevo tipo de dato: {df_delitos["tipo"].dtype}")

### Limpieza de la columna "subtipo":

In [11]:
# Verificar que los valores sean válidos
print(f"tipos de delitos: {df_delitos["subtipo"].unique()}")

tipos de delitos: ['Robo total' 'Robo automotor' 'Hurto automotor' 'Hurto total'
 'Lesiones por siniestros viales' 'Homicidios dolosos' 'Femicidios'
 'Amenazas' 'Lesiones Dolosas' 'Muertes por siniestros viales'
 'Homicidio Doloso']
tipos de delitos: ['Robo total' 'Robo automotor' 'Hurto automotor' 'Hurto total'
 'Lesiones por siniestros viales' 'Homicidios dolosos' 'Femicidios'
 'Amenazas' 'Lesiones Dolosas' 'Muertes por siniestros viales']
tipo de dato: object
nuevo tipo de dato: string


In [ ]:
# Hay dos valores, "Homicidios dolosos" y "Homicidio Doloso", que deberían ser uno solo

# Unificar las dos categorias
df_delitos["subtipo"] = df_delitos["subtipo"].replace("Homicidio Doloso", "Homicidios dolosos")

In [ ]:
# Verificar cambios
print(f"tipos de delitos: {df_delitos["subtipo"].unique()}")

In [ ]:
# Verificar tipo de dato
print(f"tipo de dato: {df_delitos["subtipo"].dtype}")

In [ ]:
# Convertir al tipo de dato correcto
df_delitos["subtipo"] = df_delitos["subtipo"].astype("string")
print(f"nuevo tipo de dato: {df_delitos["subtipo"].dtype}")

### Limpieza de la columna "uso_arma":

In [21]:
# Verificar si hay valores faltantes
print(f"cantidad de faltantes en 'uso_arma': {df_delitos["uso_arma"].isna().sum()}")

cantidad de faltantes en 'uso_arma': 0
cantidad de faltantes en 'uso_moto': 0
valores en 'uso_arma': [False  True]
valores en 'uso_moto': [False  True]
tipo de dato: bool


In [ ]:
# Convertir columnas 'uso_arma' y 'uso_moto' de SI/NO a valores booleanos
# para facilitar el análisis
bool_conversion = {"SI": True, "NO": False}
df_delitos["uso_arma"] = df_delitos["uso_arma"].apply(lambda x: bool_conversion.get(x, x))

In [ ]:
# Verifico los cambios
print(f"valores en 'uso_arma': {df_delitos["uso_arma"].unique()}")

In [ ]:
# Verifico el tipo de dato
print(f"tipo de dato: {df_delitos["uso_arma"].dtype}")

### Limpieza de la columna "uso_moto":

In [21]:
# Verificar si hay valores faltantes
print(f"cantidad de faltantes en 'uso_moto': {df_delitos["uso_moto"].isna().sum()}")

cantidad de faltantes en 'uso_arma': 0
cantidad de faltantes en 'uso_moto': 0
valores en 'uso_arma': [False  True]
valores en 'uso_moto': [False  True]
tipo de dato: bool


In [ ]:
# Convertir columnas 'uso_arma' y 'uso_moto' de SI/NO a valores booleanos
# para facilitar el análisis
bool_conversion = {"SI": True, "NO": False}
df_delitos["uso_moto"] = df_delitos["uso_moto"].apply(lambda x: bool_conversion.get(x, x))

In [ ]:
# Verifico los cambios
print(f"valores en 'uso_arma': {df_delitos["uso_moto"].unique()}")

In [ ]:
# Verifico el tipo de dato
print(f"tipo de dato: {df_delitos["uso_moto"].dtype}")

### Limpieza columna barrio

In [10]:
# Agrupar por barrio y listar las comunas únicas asociadas a cada barrio
barrios_comunas_list = df_delitos.groupby("barrio")["comuna"].unique()

# Filtrar los barrios que están asociados con más de una comuna
barrios_multiples_comunas_list = barrios_comunas_list[barrios_comunas_list.apply(len) > 1]

print("Barrios asociados con más de una comuna y sus comunas:")
for barrio, comunas in barrios_multiples_comunas_list.items():
    print(f"{barrio}: {comunas}")

Barrios asociados con más de una comuna y sus comunas:
0: <IntegerArray>
[5, 1, 2, 11, 8, 15, 4, 10, 14, 3]
Length: 10, dtype: Int8
ALMAGRO: <IntegerArray>
[5, 6, 14, 15, 3]
Length: 5, dtype: Int8
BALVANERA: <IntegerArray>
[3, 2, 5]
Length: 3, dtype: Int8
BARRACAS: <IntegerArray>
[4, 1, 3]
Length: 3, dtype: Int8
BELGRANO: <IntegerArray>
[13, 12, 14]
Length: 3, dtype: Int8
BOCA: <IntegerArray>
[4, 1]
Length: 2, dtype: Int8
BOEDO: <IntegerArray>
[5, 6]
Length: 2, dtype: Int8
CABALLITO: <IntegerArray>
[6, 5, 11, <NA>, 15, 7]
Length: 6, dtype: Int8
COGHLAN: <IntegerArray>
[12, 13]
Length: 2, dtype: Int8
COLEGIALES: <IntegerArray>
[13, 15]
Length: 2, dtype: Int8
CONSTITUCION: <IntegerArray>
[1, <NA>, 4, 3, 7]
Length: 5, dtype: Int8
FLORES: <IntegerArray>
[7, <NA>, 11, 9, 4, 6, 10]
Length: 7, dtype: Int8
FLORESTA: <IntegerArray>
[10, 11, 7]
Length: 3, dtype: Int8
FLORIDA: <IntegerArray>
[12, 13]
Length: 2, dtype: Int8
LINIERS: <IntegerArray>
[9, <NA>, 10]
Length: 3, dtype: Int8
MATADEROS: <I

### Limpieza de la columna "comuna":

Primero me aseguro que los valores de la columna sean válidos

In [4]:
# Comprobar que los valores son válidos
print(f"comunas antiguas: {df_delitos["comuna"].unique()}")

comunas antiguas: [10.0 9.0 11.0 12.0 15.0 7.0 8.0 13.0 6.0 14.0 5.0 4.0 nan 2.0 3.0 1.0
 '13' '8' '12' '7' '3' '2' '10' '4' '15' '9' '14' '1' '11' '5' '6' 'CC-08'
 'CC-09' 'CC-01 NORTE' 'CC-04' 'CC-07' 'CC-15' 'CC-02' 'CC-12' 'CC-10'
 'CC-06' 'CC-13' 'CC-05' 'CC-01 SUR' 'CC-03' 'CC-14' 'Sin geo' 'CC-11']


In [5]:
# Hay algunos valores que están escritos de manera diferente pero representan la misma comuna

# Creo un diccionario con las correcciones
dict_correciones = {
    "CC-01 NORTE": 1,
    "CC-01 SUR": 1,
    "CC-02": 2,
    "CC-03": 3,
    "CC-04": 4,
    "CC-05": 5,
    "CC-06": 6,
    "CC-07": 7,
    "CC-08": 8,
    "CC-09": 9,
    "CC-10": 10,
    "CC-11": 11,
    "CC-12": 12,
    "CC-13": 13,
    "CC-14": 14,
    "CC-15": 15,
    "Sin geo": np.nan
}

In [ ]:
# Aplicar las correciones
df_delitos["comuna"] = df_delitos["comuna"].apply(lambda comuna: dict_correciones.get(comuna, comuna))
print(f"comunas estandarizadas: {df_delitos["comuna"].unique()}")

In [ ]:
# Convertir strings numéricos a integers
df_delitos["comuna"] = pd.to_numeric(df_delitos["comuna"])
print(f"comunas solo enteros: {df_delitos["comuna"].unique()}")

In [ ]:
# Convertir tipo de dato a uno más apropiado
df_delitos["comuna"] = df_delitos["comuna"].astype("Int8")
print(f"tipo de dato: {df_delitos["comuna"].dtype}")

Ahora, compruebo si es posible rescatar los valores faltantes de las columnas por medio de los barrios

In [ ]:
# Verificar si hay registros con comunas faltantes
comunas_faltantes = df_delitos[df_delitos["comuna"].isna()]
print(f"Registros con comunas faltantes: {len(comunas_faltantes)}")

In [ ]:
# Verificar si en esos registros el barrio no falta
comunas_faltantes_con_barrio = comunas_faltantes[~comunas_faltantes["barrio"].isna()]
print(f"Registros con comunas faltantes pero con barrio: {len(comunas_faltantes_con_barrio)}")

In [ ]:
# Crear un diccionario para mapear barrios a comunas
barrio_a_comuna = df_delitos[~df_delitos["comuna"].isna()].groupby("barrio")["comuna"].first().to_dict()

In [ ]:
# Inferir la comuna basada en el barrio para aquellos registros que tienen barrio pero no tienen comuna
df_delitos.loc[df_delitos["comuna"].isna() & ~df_delitos["barrio"].isna(), "comuna"] = df_delitos["barrio"].map(barrio_a_comuna)

In [ ]:
# Verificar los cambios
comunas_faltantes_despues = df_delitos[df_delitos["comuna"].isna()]
print(f"Registros con comunas faltantes después de la inferencia: {len(comunas_faltantes_despues)}")

In [68]:
# Hallar si hay registros repetidos
print(f"delitos duplicados: {df_delitos.duplicated().sum()}")

delitos duplicados: 0


In [81]:
# Ver el tipo de dato de las columnas
df_delitos.dtypes

id-mapa     float64
anio          int64
mes          object
dia          object
fecha        object
franja      float64
tipo         object
subtipo      object
uso_arma     object
uso_moto     object
barrio       object
comuna       object
latitud      object
longitud     object
cantidad      int64
id-sum      float64
id          float64
dtype: object

In [27]:
df_delitos.loc[df_delitos["barrio"] == "ALMAGRO", "comuna"].value_counts()

df_delitos.

comuna
5     41576
6        12
15        9
14        4
3         1
Name: count, dtype: Int64